-----
1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какой график (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого, например, 1 к 1000).
p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно).
Допустим, у нас две модели:

первая помечает 100 объектов как класс 1, но TP = 90
вторая помечает 1000 объектов как класс 1, но TP такой же - 90
Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

----

In [56]:
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [7]:
FILE_NAME = 'train_case2.csv'
FILE_PATH = Path.cwd() / FILE_NAME

df = pd.read_csv(FILE_PATH, sep=';')

In [9]:
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


----
**1-2 задания:**

In [88]:
cv = 5
target = 'cardio'


model_log = LogisticRegression()
model_forest = RandomForestClassifier()
model_knn = KNeighborsClassifier()
X_train = df.drop(columns=target)
y_train = df[target]

results = pd.DataFrame({'algorithm':[],
                        'precision':[],
                        'recall':[],
                        'roc_auc':[],
                        'f_score': []})

In [81]:
%%time
cv_log = cross_validate(model_log, X_train, y_train, cv=cv,
                           scoring=['recall', 'precision', 'roc_auc', 'f1'])
results = results.append({'algorithm':'LogisticRegression',
                        'precision':cv_log['test_recall'].mean(),
                        'recall':cv_log['test_precision'].mean(),
                        'roc_auc':cv_log['test_roc_auc'].mean(),
                        'f_score': cv_log['test_f1'].mean()},
                        ignore_index=True)
print(f'CV with {cv} folds for LogReg')

CV with 5 folds for LogReg
Wall time: 2.1 s


In [82]:
%%time
cv_forest = cross_validate(model_forest, X_train, y_train, cv=cv,
                           scoring=['recall', 'precision', 'roc_auc', 'f1'])
results = results.append({'algorithm':'RandomForestClassifier',
                        'precision':cv_forest['test_recall'].mean(),
                        'recall':cv_forest['test_precision'].mean(),
                        'roc_auc':cv_forest['test_roc_auc'].mean(),
                        'f_score': cv_forest['test_f1'].mean()},
                        ignore_index=True)
print(f'CV with {cv} folds for Random Forest')

CV with 5 folds for Random Forest
Wall time: 55.7 s


In [85]:
%%time
cv_knn = cross_validate(model_knn, X_train, y_train, cv=cv,
                           scoring=['recall', 'precision', 'roc_auc', 'f1'])
results = results.append({'algorithm':'KNeighborsClassifier',
                        'precision':cv_knn['test_recall'].mean(),
                        'recall':cv_knn['test_precision'].mean(),
                        'roc_auc':cv_knn['test_roc_auc'].mean(),
                        'f_score': cv_knn['test_f1'].mean()},
                        ignore_index=True)
print(f'CV with {cv} folds for KNN')

CV with 5 folds for KNN
Wall time: 27.9 s


----
**3 задание:**

In [86]:
results

,algorithm,precision,recall,roc_auc,f_score
0,LogisticRegression,0.659739,0.710336,0.756252,0.683949
1,RandomForestClassifier,0.619387,0.725723,0.772783,0.630131
2,KNeighborsClassifier,0.505060,0.567357,0.524908,0.443179


----
**4 задание:**

При выбранных гиперпараметрах моделей KNN оказалась хуже остальных по всем параметрам (кроме веремни обучения относительно случайного леса), поэтому ее из дальнейшего рассмотрения можно отбросить.

Априори сказать какая из ситуаций оказалась лучше нельзя:
* если необходим больший охват объектов первого класса (в задачах медицинской тематики обычно это и нужно), то стоит выбрать логистическую регрессию;
* если требуется более высокая точность модели, то стоит выбрать модель, построенную на алгоритме случайного леса;
* если требуется достаточно быстрое и обучение, и предсказание результатов, то выгоднее брать модель логистической регрессии.

----
**5 задание:**

В случае дисбаланса классов при использовании ROC кривой будут учтены оба класса одинаково (не важно в какую сторону идет перевес), то есть можно наблюдать более стабильную картину, только лишь отраженную относительно главной диагонали.

При использовании PR кривой акцент отрисовки будет сделан на меньшем по численности классе, в результате чего даже визуально будет достаточно четко видно, при каком пороге график начинает резко падать, что позволяет легче его подобрать без особой потери `precision` или `recall`, максимизируя другую метрику.